In [20]:
import numpy as np
import csv
import torch
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import cm

with open('temp_co2_data.csv') as data_file:
    data = list(csv.reader(data_file))[1:]
    in_features = torch.tensor([[float(item) for item in line[2:]] for line in data])
    out_features = torch.tensor([[float(line[1])] for line in data])

design_matrix = torch.tensor([[1, *line] for line in in_features])
weights_linear_algebra = torch.linalg.lstsq(design_matrix, out_features, driver='gels').solution[:, 0]

x_mean = in_features.mean(0)
y_mean = out_features.mean()

x_standard_deviation = in_features.std(0)
y_standard_deviation = out_features.std()

in_features = (in_features - x_mean) / x_standard_deviation
out_features = (out_features - y_mean) / y_standard_deviation

design_matrix = torch.tensor([[1, *line] for line in in_features])
weights = torch.rand(3, 1) - 0.5 * torch.ones(3, 1)

learning_rate = 0.5
epochs = 5000

all_weights = []
all_losses = []

loss = 0

for x in np.arange(-1., 1. + 2/18., 2/18.):
    for y in np.arange(-1., 1. + 2/18., 2/18.):
        for z in np.arange(-1., 1. + 2/18., 2/18.):
            weights = (torch.rand(3, 1) - 0.5) * 1000 + 0
            weights[0] = x
            weights[1] = y
            weights[2] = z
            weights *= 2
            target_estimates = design_matrix.mm(weights)

            loss = (target_estimates - out_features).pow(2).sum() / design_matrix.size(0)

            all_weights.append([i for i in [*weights.squeeze(1)]])
            all_losses.append(loss)

# for epoch in range(epochs):
#     weights = (torch.rand(3, 1) - 0.5) * 2
#     target_estimates = design_matrix.mm(weights)
# 
#     loss = (target_estimates - y_features).pow(2).sum() / design_matrix.size(0)
#     # print(f'epoch: {epoch + 1}, current loss: {loss.item()}')
#     # weights = weights.squeeze(1)
#     # weights[1:] = weights[1:] * y_standard_deviation / x_standard_deviation
#     # weights[0] = weights[0] * y_standard_deviation + y_mean - weights[1:] @ x_mean
#     # all_weights.append([i for i in [*weights]])
# 
#     all_weights.append([i for i in [*weights.squeeze(1)]])
#     all_losses.append(loss)

all_weights = torch.tensor(all_weights)

fig = plt.figure(dpi=200)
ax = plt.axes(projection='3d')

# ax.scatter(all_weights[:, 0], all_weights[:, 2], all_losses, c=all_weights[:, 1], cmap=cm.coolwarm)
ax.scatter(all_weights[:, 0], all_weights[:, 1], all_weights[:, 2], c=all_losses, cmap=cm.coolwarm)
ax.set_xlabel('Intercept')
ax.set_ylabel('Solar Irradiance Coefficient')
ax.set_zlabel('Loss')

plt.show()

weights = weights.squeeze(1)
weights[1:] = weights[1:] * y_standard_deviation / x_standard_deviation
weights[0] = weights[0] * y_standard_deviation + y_mean - weights[1:] @ x_mean

print(
    f'The least-squares regression plane:\n'
    f'\tfound by the neural net is: y = {weights[0]:.3f} + {weights[1]:.3f} * x1 + {weights[2]:.3f} * x2\n'
    f'\tusing linear algebra:\t\ty = {weights_linear_algebra[0]:.3f} + {weights_linear_algebra[1]:.3f} * x1 + '
    f'{weights_linear_algebra[2]:.3f} * x2\n'
    f'Learning rate: {learning_rate}\n'
)


The least-squares regression plane:
	found by the neural net is: y = -122898.273 + 2.393 * x1 + 89.394 * x2
	using linear algebra:		y = -11371.969 + 1.147 * x1 + 8.047 * x2
Learning rate: 0.5
